In [1]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
data_root_path = "../data/Challenge/"
embedding_path = "../../"
KG_root_path = "../data/Challenge/entity"
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title', 'entity','vert'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
            'batch_size': 32,
             'max_entity_num':5}
model_config = {
        'news_encoder':0,
        'popularity_user_modeling':True,
        'rel':True,
        'ctr':True,
        'content':True,
        'rece_emb':True,
        'activity':True

    }

In [4]:
News = NewsContent(config)

TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

19779
2472
2473


In [5]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

19779
19916
2473
2472


In [6]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)
entity_embedding_matrix, have_word2 = load_matrix(embedding_path, News.entity_dict)

In [7]:
train_loader = DataLoader(TrainDataset(News, TrainUsers, train_sess, train_user_id, train_buckets, train_label), config['batch_size'])
test_user_data = UserDataset(News,TestUsers), config['batch_size']
val_user_data = UserDataset(News,ValidUsers)
news_data = NewsDataset(News)

In [8]:
for x, y in train_loader:
    print(type(x[1]))
    break
    
    

<class 'torch.Tensor'>


In [9]:
from torch.optim import Adam

model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = \
create_pe_model(config, model_config, News, title_word_embedding_matrix, entity_embedding_matrix, device)

model.to(device)
for name, module in model.named_children():
    module.to(device)
    print(f'Moved {name} to {device}')



Moved news_encoder to cuda
Moved bias_news_encoder to cuda
Moved pop_aware_user_encoder to cuda
Moved bias_scorer to cuda
Moved activity_gater to cuda
Moved time_embedding_layer to cuda
Moved time_distributed1 to cuda
Moved time_distributed2 to cuda
Moved time_distributed3 to cuda
Moved scaler to cuda
Moved softmax to cuda


In [10]:
for name, param in model.named_parameters():
    print(f'{name}: {param.device}')

news_encoder.word_embedding_layer.weight: cuda:0
news_encoder.entity_embedding_layer.weight: cuda:0
news_encoder.attention.WQ.weight: cuda:0
news_encoder.attention.WK.weight: cuda:0
news_encoder.attention.WV.weight: cuda:0
news_encoder.attentive_pool.dense_tanh.weight: cuda:0
news_encoder.attentive_pool.dense_tanh.bias: cuda:0
news_encoder.attentive_pool.dense_flat.weight: cuda:0
news_encoder.attentive_pool.dense_flat.bias: cuda:0
news_encoder.title_encoder.conv.weight: cuda:0
news_encoder.title_encoder.conv.bias: cuda:0
news_encoder.title_encoder.attentivePool.dense_tanh.weight: cuda:0
news_encoder.title_encoder.attentivePool.dense_tanh.bias: cuda:0
news_encoder.title_encoder.attentivePool.dense_flat.weight: cuda:0
news_encoder.title_encoder.attentivePool.dense_flat.bias: cuda:0
news_encoder.vert_encoder.embedding_layer.weight: cuda:0
bias_news_encoder.word_embedding_layer.weight: cuda:0
bias_news_encoder.entity_embedding_layer.weight: cuda:0
bias_news_encoder.attention.WQ.weight: cud

In [11]:
from tqdm import tqdm
val_metrics_epoch = []
num_epochs = 1
# Step 2: Create your Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

loss_fn = nn.CrossEntropyLoss()

# Step 3: Iterate over the data for the number of epochs
for epoch in range(num_epochs):

# Step 4: Iterate over each batch of data and compute the scores using the forward pass of the network
    model.train()
    for x, y in tqdm(train_loader):
        optimizer.zero_grad()
        x= [i.to(device) for i in x]
        y = y.to(device)
        out = model(x)
    
        # Step 5: Compute the lambda gradient values for the pairwise loss (spedup) with the compute_lambda_i method on the scores and the output labels
        loss = loss_fn(out, y)

        # Step 6: Bacward from the scores with the use of the lambda gradient values
        if loss is not None:
            # torch.autograd.backward(out, loss)
            loss.backward()
            
            # Step 7: Update the weights using the optimizer
            optimizer.step()

    model.eval()

    val_metrics = eval_model(model_config, News, user_encoder, val_impressions, val_user_data, val_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, device)
    
    print("epoch: {}, metrics: {}".format(epoch, val_metrics))
    val_metrics_epoch.append(val_metrics)



  0%|          | 0/2472 [00:00<?, ?it/s]C:\Users\ivvbr\OneDrive - Nimble Institute\Desktop\MSc AI\Recommender Systems\RecSys-PP-Rec\RecSys-PP-Rec\Code\PEGenerator.py:266: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  user_feature = torch.IntTensor([self.News.fetch_news(clicked_ids)])
100%|██████████| 2472/2472 [00:30<00:00, 82.07it/s]

epoch: 0, metrics: [0.6845547538907755, 0.46860510251619875, 0.5237957357439255, 0.5747252782663749]


In [ ]:
test_metrics = eval_model(model_config, News, user_encoder, test_impressions, test_user_data, test_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler)

In [ ]:
from tqdm import trange
model.eval()
rankings = []
AUC = []
MRR = []
nDCG5 = []
nDCG10 =[]
for i in trange(len(val_impressions)):
    docids = val_impressions[i]['docs']
    docids = np.array(docids)
    bucket = val_impressions[i]['tsp']
    
    publish_time = News.news_publish_bucket2[docids]

    if model_config['rel']:        
        uv = user_encoder(val_user_data.__getitem__(val_userids[i]))
        nv = news_encoder(torch.IntTensor(News.fetch_news(docids)))
        rel_score = torch.matmul(nv, uv[0])
        predicted_activity_gate = activity_gater(uv)
        predicted_activity_gate = predicted_activity_gate[:,0]

    else:
        rel_score = 0
    
    if model_config['content'] and model_config['rece_emb']:
        bias_vecs = bias_news_encoder(torch.IntTensor(News.fetch_news(docids)))
        publish_time = bucket - publish_time
        arg = publish_time < 0
        publish_time[arg] = 0
        publish_bucket = compute_Q_publish(publish_time)
        time_emb = time_embedding_layer.weight[publish_bucket]
        bias_vecs = torch.cat([bias_vecs,time_emb], axis=-1)
        bias_score = bias_content_scorer(bias_vecs)
        bias_score = bias_score[:,0]
    else:
        bias_score = 0


    if model_config['activity']:
        gate = predicted_activity_gate
    else:
        gate = 0.5
    
    if model_config['ctr']:
        ctr = torch.FloatTensor(fetch_ctr_dim1(News,docids,bucket,FLAG_CTR))
    else:
        ctr = 0

    score = gate*rel_score + (1-gate)*(ctr*scaler.scaler[0] + bias_score)
    score = score.detach().numpy()
    # print(score)

    labels = val_impressions[i]['labels']
    labels = np.array(labels)
    
    auc = my_auc(labels,score)
    mrr = mrr_score(labels,score)
    ndcg5 = ndcg_score(labels,score,k=5)
    ndcg10 = ndcg_score(labels,score,k=10)

    # print(auc, mrr, ndcg5, ndcg10)

    AUC.append(auc)
    MRR.append(mrr)
    nDCG5.append(ndcg5)
    nDCG10.append(ndcg10)
    # return AUC, MRR, nDCG5, nDCG10

    # rankings.append(score)
    

AUC = np.array(AUC)
MRR = np.array(MRR)
nDCG5 = np.array(nDCG5)
nDCG10 = np.array(nDCG10)

AUC = AUC.mean()
MRR = MRR.mean()
nDCG5 = nDCG5.mean()
nDCG10 = nDCG10.mean()

## Finish this code

In [ ]:
cold = []
for TOP_COLD_NUM in [0,1,3,5,]:
    g = evaluate_cold_users(rankings,test_impressions,TestUsers.click,TOP_COLD_NUM)
    cold.append(g)
diversity = []
for TOP_DIVERSITY_NUM in range(1,11):
    div_top = evaluate_diversity_topic_all(TOP_DIVERSITY_NUM,rankings,test_impressions,News,TestUsers)
    div_ilxd = evaluate_density_ILxD(TOP_DIVERSITY_NUM,rankings,test_impressions,news_scoring)
    diversity.append([div_top,div_ilxd])

In [ ]:
print("val metrics", val_metrics_epoch)
print("test metrics", performance)
print("cold", cold)
print("diversity", diversity)

results = {"val metrics": val_metrics_epoch, 
           "test metrics": performance,
           "cold": cold,
           "diversity": diversity}

import json
with open('results.json', 'w') as f:
    json.dump(results, f)

## Finished
## ---

In [ ]:
# model.compile(loss=['categorical_crossentropy'],
#                   optimizer=Adam(lr=0.0001), 
#                   metrics=['acc'])

In [ ]:
# for i in range(10):

#     model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = create_pe_model(config,model_config,News,title_word_embedding_matrix,News.entity_embedding)
#     model.fit_generator(train_generator,epochs=2)
#     news_scoring = news_encoder.predict_generator(news_generator,verbose=True)
#     user_scoring = user_encoder.predict_generator(test_user_generator,verbose=True)
#     val_user_scoring = user_encoder.predict_generator(val_user_generator,verbose=True)


#     news_bias_vecs = bias_news_encoder.predict_generator(news_generator,verbose=True)

#     if model_config['content'] and not model_config['rece_emb']:
#         bias_candidate_score = bias_content_scorer.predict(news_bias_vecs,batch_size=32,verbose=True)
#         bias_candidate_score = bias_candidate_score[:,0]
#     else:
#         bias_candidate_score = 0

#     ctr_weight = scaler.get_weights()[0][0,0]
#     time_embedding_matrix = time_embedding_layer.get_weights()[0]
    
#     predicted_activity_gates = activity_gater.predict(user_scoring,verbose=True)
#     predicted_activity_gates = predicted_activity_gates[:,0]
    
#     val_predicted_activity_gates = activity_gater.predict(val_user_scoring,verbose=True)
#     val_predicted_activity_gates = val_predicted_activity_gates[:,0]
    
#     rankings = news_ranking(model_config,ctr_weight,predicted_activity_gates,user_scoring,news_scoring,
#                                 bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                 News,test_impressions)
    
#     val_rankings = news_ranking(model_config,ctr_weight,val_predicted_activity_gates,val_user_scoring,news_scoring,
#                                bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                News,val_impressions)
    
#     performance = evaluate_performance(rankings,test_impressions)
#     val_performance = evaluate_performance(val_rankings,val_impressions)

#     cold = []
#     for TOP_COLD_NUM in [0,1,3,5,]:
#         g = evaluate_cold_users(rankings,test_impressions,TestUsers.click,TOP_COLD_NUM)
#         cold.append(g)
#     diversity = []
#     for TOP_DIVERSITY_NUM in range(1,11):
#         div_top = evaluate_diversity_topic_all(TOP_DIVERSITY_NUM,rankings,test_impressions,News,TestUsers)
#         div_ilxd = evaluate_density_ILxD(TOP_DIVERSITY_NUM,rankings,test_impressions,news_scoring)
#         diversity.append([div_top,div_ilxd])


AttributeError: 'NewsContent' object has no attribute 'entity_embedding'